In [2]:
sc

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res0: org.apache.spark.SparkContext = org.apache.spark.SparkContext@7b37262d


In [3]:
import org.apache.spark.sql.{DataFrame,Column}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.{DataFrame, Column}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._


In [4]:
def mapMonth = udf(
    (month: String) => month match {
        case "January" => "01"
        case "February" => "02"
        case "March" => "03"
        case "April" => "04"
        case "May" => "05"
        case "June" => "06"
        case "July" => "07"
        case "August" => "08"
        case "September" => "09"
        case "October" => "10"
        case "November" => "11"
        case "December" => "12"
})

def mapDay = udf(
    (day: String) => day match {
        case "1" => "01"
        case "2" => "02"
        case "3" => "03"
        case "4" => "04"
        case "5" => "05"
        case "6" => "06"
        case "7" => "07"
        case "8" => "08"
        case "9" => "09"
        case _ => day
})


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

mapMonth: org.apache.spark.sql.expressions.UserDefinedFunction
mapDay: org.apache.spark.sql.expressions.UserDefinedFunction


In [5]:
 def preProcessingDF(data: DataFrame):DataFrame = {
     (data
      .select("_id","createdAt","text","title")
      .withColumn("id", col("_id.$oid"))
      .drop("_id")
      .withColumn("month-day",
                  split(col("createdAt"),",")
                 )
      .withColumn("month",
                   element_at(
                       split(element_at(col("month-day"),1)," "),1)
                  )
       .withColumn("day",
                   element_at(
                       split(element_at(col("month-day"),1)," "),2)
                  )
       .withColumn("year",
                   element_at(
                       col("month-day"),2)
                  )
       .drop("month-day","createdAt")
       .withColumn("Day",mapDay(col("day")))
       .withColumn("Month",mapMonth(col("month")))
       .withColumn("Year",trim(col("year")))
       .withColumn("Date",concat(col("day"), lit("/"),col("month"),lit("/"),col("Year")))
       .withColumn("Date",to_date(col("Date"),"dd/MM/yyyy"))
       .drop("Month","Day","Year")
      )

}


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

preProcessingDF: (data: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


In [6]:
val s3_path = "s3://trab4-bucket/datasets/eiffel-tower-reviews.json"

val data = ( spark
            .read
            .option("inferSchema","true")
            .json(s3_path) ) 

data.printSchema()
data.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3_path: String = s3://trab4-bucket/datasets/eiffel-tower-reviews.json
data: org.apache.spark.sql.DataFrame = [_id: struct<$oid: string>, author: struct<helpfulVotes: string, level: string ... 4 more fields> ... 6 more fields]
root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- author: struct (nullable = true)
 |    |-- helpfulVotes: string (nullable = true)
 |    |-- level: string (nullable = true)
 |    |-- memberSince: string (nullable = true)
 |    |-- postForum: string (nullable = true)
 |    |-- ratings: string (nullable = true)
 |    |-- reviews: string (nullable = true)
 |-- bubbleCount: long (nullable = true)
 |-- collectedAt: struct (nullable = true)
 |    |-- $date: string (nullable = true)
 |-- createdAt: string (nullable = true)
 |-- query: string (nullable = true)
 |-- text: string (nullable = true)
 |-- title: string (nullable = true)

+--------------------+--------------------+-----------+--------------------+------------+------------+-

In [7]:
val data_processed = preProcessingDF(data)
data_processed.write.parquet("s3://trab4-bucket/datasets/torre-eifel.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

data_processed: org.apache.spark.sql.DataFrame = [text: string, title: string ... 2 more fields]
